In [ ]:
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
import scipy.stats
import os
import scipy.io as sio
import regex as re
from collections import Counter, defaultdict
#from pylab import *
#import matplotlib.pyplot as plt
import sys 
#%matplotlib inline

from skbio import TabularMSA, DNA
from skbio.alignment import local_pairwise_align_ssw

CONST_A = 0
CONST_C = 1
CONST_G = 2
CONST_T = 3

CONST_NT_MAP = ['A', 'C', 'G', 'T']

def distance(astring, bstring) :
    distance = 0
    
    limit = len(astring)
    diff = len(bstring) - len(astring)
    if len(bstring) < len(astring) :
        limit = len(bstring)
        diff = len(astring) - len(bstring)
    
    for i in range(limit) :
        if astring[i] != bstring[i] :
            distance += 1
    return distance + diff

def reverse_complement(seq) :
    rc_seq = ''
    for i in range(0, len(seq)) :
        if seq[i] == 'A' :
            rc_seq = 'T' + rc_seq
        elif seq[i] == 'C' :
            rc_seq = 'G' + rc_seq
        elif seq[i] == 'G' :
            rc_seq = 'C' + rc_seq
        elif seq[i] == 'T' :
            rc_seq = 'A' + rc_seq
    return rc_seq

def get_hamming_neighbor_1(seq, seq_map, start_r, end_r) :
    for i in range(start_r, end_r) :
        for base1 in CONST_NT_MAP :
            mut_seq = seq[:i] + base1 + seq[i+1:]
            if mut_seq in seq_map :
                return mut_seq
    return None

def get_hamming_neighbor_2(seq, seq_map, start_r, end_r) :
    for i in range(start_r, end_r) :
        for j in range(i + 1, end_r) :
            for base1 in CONST_NT_MAP :
                for base2 in CONST_NT_MAP :
                    mut_seq = seq[:i] + base1 + seq[i+1:j] + base2 + seq[j+1:]
                    if mut_seq in seq_map :
                        return mut_seq
    return None

In [4]:
r1_rna = 'r1_rna.fq'
r2_rna = 'r2_rna.fq'
i1_rna = 'i1_rna.fq'

distal_utr = 'gtgccttctagttgccagccatctgttgtttgcccctcccccgtgccttccttgaccctggaaggtgccactcccactgtcctttcctaataaaatgaggaaattgcatcgcattgtctgagtaggtgtcattctattctggggggtggggtggggcaggacagcaaggg'.upper()
print(distal_utr)

distal_utr_DNA = DNA(distal_utr)

GTGCCTTCTAGTTGCCAGCCATCTGTTGTTTGCCCCTCCCCCGTGCCTTCCTTGACCCTGGAAGGTGCCACTCCCACTGTCCTTTCCTAATAAAATGAGGAAATTGCATCGCATTGTCTGAGTAGGTGTCATTCTATTCTGGGGGGTGGGGTGGGGCAGGACAGCAAGGG


In [5]:
dna_file = pd.read_csv('apa_sym_prx_dna_hamming_20160825.csv',sep=',')

dna_barcode_list = list(dna_file.barcode)
dna_sequence_list = list(dna_file.sequence)

dna_barcode_map = {}
dna_sequence_map = {}

for i in range(0, len(dna_barcode_list)) :
    dna_barcode_map[dna_barcode_list[i]] = dna_barcode_list[i]
    dna_sequence_map[dna_barcode_list[i]] = dna_sequence_list[i]

In [6]:
print(len(dna_barcode_map))

2434388


In [7]:
f = {}
f[0] = open(r1_rna,'r')
f[1] = open(r2_rna,'r')

i1 = open(i1_rna, 'r')


start_from_count = 0

file_action = 'w'
if start_from_count > 0 :
    file_action = 'a'


head, seq, pr, q = ({} for i in range(4))
count = 0

mapped_umi_map = {}

total_matched_count = 0
total_unique_umi_count = 0
total_mapped_count = 0

matched_on_dict_count = 0
matched_on_hamming1_count = 0
matched_on_hamming2_count = 0

print('Processing RNA reads.')

out = open('apa_sym_prx_mapped_rna_20160916.csv', file_action)
out.write('barcode,umi,rna_read,polya_pos,align_score\n')

while True:
    for i in range(2):
        head[i] = f[i].readline()[:-1]
        seq[i] = f[i].readline()[:-1]
        pr[i] = f[i].readline()[:-1]
        q[i] = f[i].readline()[:-1]
        
    headi = i1.readline()[:-1]
    seqi = i1.readline()[:-1]
    pri = i1.readline()[:-1]
    qi = i1.readline()[:-1]
    
    if len(seq[0]) == 0:
        break # End of File
    
    if count < start_from_count :
        count += 1
        continue
    
    barcode = seq[0]
    umi = seqi
    rna_seq = reverse_complement(seq[1])

    barcode_key = barcode
    matched = False
    if barcode in dna_barcode_map :
        matched = True
        matched_on_dict_count += 1
    else :
        barcode_h1 = get_hamming_neighbor_1(barcode, dna_barcode_map, 0, 20)
        if barcode_h1 != None :
            matched = True
            barcode_key = barcode_h1
            matched_on_hamming1_count += 1
    
    if matched == True :
        total_matched_count += 1
        
        #if barcode_key not in mapped_umi_map :
        #    mapped_umi_map[barcode_key] = {}
        
        is_dup_umi = False
        
        #Exact match UMI
        #if umi in mapped_umi_map[barcode_key] :
        #    is_dup_umi = True
        
        if is_dup_umi == False :
            total_unique_umi_count += 1
            
            #mapped_umi_map[barcode_key][umi] = umi
            
            rna_seq_DNA = DNA(rna_seq)
            
            polya_pos = -1
            score = -10000
            
            ref_seq = dna_sequence_map[barcode_key]
            candidate_tuple = local_pairwise_align_ssw(DNA(ref_seq),rna_seq_DNA,score_filter=30)

            if candidate_tuple != None :
                _, score, start_end_positions = candidate_tuple
                polya_pos = start_end_positions[0][1] + (50 - 1 - start_end_positions[1][1]) #+ 73
                total_mapped_count += 1
            else :
                candidate_tuple = local_pairwise_align_ssw(distal_utr_DNA,rna_seq_DNA,score_filter=30)
                if candidate_tuple != None :
                    _, score, start_end_positions = candidate_tuple
                    polya_pos = start_end_positions[0][1] + (50 - 1 - start_end_positions[1][1]) + 207#+ 280
                    total_mapped_count += 1
            
            if polya_pos != -1 :
                out.write(barcode_key)
                out.write(',' + umi)
                out.write(',' + rna_seq)
                out.write(',' + str(polya_pos))
                out.write(',' + str(score))
                out.write('\n')
        
    if count % 1000000 == 0:
        print('Count: ' + str(count))
        print('RNA reads matched against barcodes: ' + str(total_matched_count))
        print('Matched RNA reads with unique UMI: ' + str(total_unique_umi_count))
        print('Unique UMI reads mapped to reference: ' + str(total_mapped_count))
        
        print('Matched on dictionary: ' + str(matched_on_dict_count))
        print('Matched on hamming 1: ' + str(matched_on_hamming1_count))
        print('Matched on hamming 2: ' + str(matched_on_hamming2_count))
    count += 1
    
print('COMPLETE')
print('RNA reads matched against barcodes: ' + str(total_matched_count))
print('Matched RNA reads with unique UMI: ' + str(total_unique_umi_count))
print('Unique UMI reads mapped to reference: ' + str(total_mapped_count))

print('Matched on dictionary: ' + str(matched_on_dict_count))
print('Matched on hamming 1: ' + str(matched_on_hamming1_count))
print('Matched on hamming 2: ' + str(matched_on_hamming2_count))

out.close()

f[0].close()
f[1].close()
i1.close()#18:05

Count: 229000000
RNA reads matched against barcodes: 133115150
Matched RNA reads with unique UMI: 133115150
Unique UMI reads mapped to reference: 95623090
Matched on dictionary: 125573494
Matched on hamming 1: 7541656
Matched on hamming 2: 0
Count: 230000000
RNA reads matched against barcodes: 133697004
Matched RNA reads with unique UMI: 133697004
Unique UMI reads mapped to reference: 96052129
Matched on dictionary: 126122821
Matched on hamming 1: 7574183
Matched on hamming 2: 0
Count: 231000000
RNA reads matched against barcodes: 134280707
Matched RNA reads with unique UMI: 134280707
Unique UMI reads mapped to reference: 96470420
Matched on dictionary: 126673512
Matched on hamming 1: 7607195
Matched on hamming 2: 0
Count: 232000000
RNA reads matched against barcodes: 134863415
Matched RNA reads with unique UMI: 134863415
Unique UMI reads mapped to reference: 96877138
Matched on dictionary: 127223817
Matched on hamming 1: 7639598
Matched on hamming 2: 0
Count: 233000000
RNA reads match